In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_huggingface import HuggingFaceEndpoint
from langchain.tools import Tool
from langchain.chat_models import ChatOllama
from langchain_community.utilities import GoogleSerperAPIWrapper
from dotenv import load_dotenv


import getpass
import os

In [ ]:
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

api_key = os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = api_key

## Using Langchain Agent

LLM: https://python.langchain.com/docs/integrations/llms/

AgentType: https://github.com/langchain-ai/langchain/discussions/12888

In [ ]:
def multiply_numbers(x, y):
    return x * y

math_tool = Tool(
    name="MultiplyNumbers",
    func=lambda inputs: multiply_numbers(*inputs),
    description="Use this tool to multiply two numbers.",
    return_direct=True
)

llm = ChatOllama(model="llama3")

agent = initialize_agent(
    tools=[math_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

response = agent.invoke("What is 7 multiplied by 6?")
print(response)

In [ ]:
MODEL = "llama3"

# Google Search Tool
search = GoogleSerperAPIWrapper()

# Initialize LLM
llm = ChatOllama(model=MODEL)

# Define the tool
search_tool = Tool(
    name="Google Search",
    func=search.run,
    description="Use this tool when you need to search for real-time information from Google.",
    return_direct=True
)

# List of tools the agent can use
tools = [search_tool]

agent = initialize_agent(
    tools=tools,                    # Get access to the tool, use tool based on descriptions
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,     # Allows the agent to retry when parsing fails
    return_intermediate_steps=True,
    max_iterations=3                # limit the number of iterations
)

# Example query
response = agent.invoke("Is bill moyers dead yet? How old is he")

print(response)